# Variable Elimination


In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('B', 'A'), ('E', 'A'), ('A', 'S'), ('A', 'D')])

# Define the Conditional Probability Distributions (CPDs)
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.99], [0.01]])
cpd_e = TabularCPD(variable='E', variable_card=2, values=[[0.98], [0.02]])
cpd_a = TabularCPD(variable='A', variable_card=2,
                   values=[[0.999, 0.71, 0.06, 0.05],
                           [0.001, 0.29, 0.94, 0.95]],
                   evidence=['B', 'E'], evidence_card=[2, 2])
cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.1],
                           [0.05, 0.9]],
                   evidence=['A'], evidence_card=[2])
cpd_d = TabularCPD(variable='D', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['A'], evidence_card=[2])

# Add CPDs to the model
model.add_cpds(cpd_b, cpd_e, cpd_a, cpd_s, cpd_d)

# Check if the model is consistent with the CPDs
assert model.check_model()

# Initialize the Variable Elimination inference
inference = VariableElimination(model)

# Computing the marginal probability of a variable
marginal_prob_s = inference.query(variables=['S'])
marginal_prob_d = inference.query(variables=['D'])

print("Marginal Probability of Sophia calls:")
print(marginal_prob_s)

print("\nMarginal Probability of David calls:")
print(marginal_prob_d)

Marginal Probability of Sophia calls:
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.9363 |
+------+----------+
| S(1) |   0.0637 |
+------+----------+

Marginal Probability of David calls:
+------+----------+
| D    |   phi(D) |
+======+==========+
| D(0) |   0.9379 |
+------+----------+
| D(1) |   0.0621 |
+------+----------+


# code 2

In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('B', 'A'), ('E', 'A'), ('A', 'S'), ('A', 'D')])

# Define the Conditional Probability Distributions (CPDs)
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.99], [0.01]])
cpd_e = TabularCPD(variable='E', variable_card=2, values=[[0.98], [0.02]])
cpd_a = TabularCPD(variable='A', variable_card=2,
                   values=[[0.999, 0.71, 0.06, 0.05],
                           [0.001, 0.29, 0.94, 0.95]],
                   evidence=['B', 'E'], evidence_card=[2, 2])
cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.1],
                           [0.05, 0.9]],
                   evidence=['A'], evidence_card=[2])
cpd_d = TabularCPD(variable='D', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['A'], evidence_card=[2])

# Add CPDs to the model
model.add_cpds(cpd_b, cpd_e, cpd_a, cpd_s, cpd_d)

# Check if the model is consistent with the CPDs
assert model.check_model()

# Initialize the Variable Elimination inference
inference = VariableElimination(model)

# Function to perform exact inference using Variable Elimination
def exact_inference(query_variables):
    # Compute the exact marginal probabilities
    result = inference.query(variables=query_variables)
    return result

# Perform exact inference for Sophia calls (S) and David calls (D)
query_variables = ['S', 'D']
result = exact_inference(query_variables)

# Print the exact marginal probabilities
for var in query_variables:
    print(f"Exact Marginal Probability of {var}:")
    print(result[var])


Exact Marginal Probability of S:


TypeError: 'DiscreteFactor' object is not subscriptable

# code 3


In [3]:
import numpy as np
from functools import reduce

class Factor:
    def __init__(self, variables, table):
        self.variables = variables
        self.table = table

    def sum_out(self, variable):
        axis = self.variables.index(variable)
        new_variables = [v for v in self.variables if v != variable]
        new_table = np.sum(self.table, axis=axis)
        return Factor(new_variables, new_table)

    def multiply(self, other):
        common_vars = list(set(self.variables) & set(other.variables))
        new_variables = list(set(self.variables + other.variables))
        new_shape = [self.table.shape[self.variables.index(var)] if var in self.variables else other.table.shape[other.variables.index(var)] for var in new_variables]
        
        self_broadcast = np.reshape(self.table, [self.table.shape[self.variables.index(var)] if var in self.variables else 1 for var in new_variables])
        other_broadcast = np.reshape(other.table, [other.table.shape[other.variables.index(var)] if var in other.variables else 1 for var in new_variables])
        
        new_table = self_broadcast * other_broadcast
        return Factor(new_variables, new_table)

    def reduce(self, evidence):
        indices = tuple(slice(None) if var not in evidence else evidence[var] for var in self.variables)
        new_variables = [v for v in self.variables if v not in evidence]
        new_table = self.table[indices]
        return Factor(new_variables, new_table)

def variable_elimination(factors, query, evidence):
    def eliminate_var(factors, variable):
        factors_with_var = [f for f in factors if variable in f.variables]
        factors_without_var = [f for f in factors if variable not in f.variables]
        new_factor = reduce(lambda f1, f2: f1.multiply(f2), factors_with_var).sum_out(variable)
        return factors_without_var + [new_factor]

    for var in evidence:
        factors = [factor.reduce(evidence) for factor in factors]

    elimination_order = [var for var in set(reduce(lambda x, y: x + y, [factor.variables for factor in factors])) if var not in query and var not in evidence]
    for variable in elimination_order:
        factors = eliminate_var(factors, variable)

    final_factor = reduce(lambda f1, f2: f1.multiply(f2), factors)
    normalization_constant = np.sum(final_factor.table)
    final_factor.table = final_factor.table / normalization_constant
    
    query_index = tuple(evidence[var] if var in evidence else slice(None) for var in final_factor.variables)
    return final_factor.table[query_index]

# Example usage:

# Define the factors for the network
# Assuming a simple network for demonstration

# P(A)
A = Factor(['A'], np.array([0.2, 0.8]))

# P(B|A)
B_given_A = Factor(['A', 'B'], np.array([[0.5, 0.5], [0.7, 0.3]]))

# P(C|A)
C_given_A = Factor(['A', 'C'], np.array([[0.4, 0.6], [0.9, 0.1]]))

# P(D|B,C)
D_given_BC = Factor(['B', 'C', 'D'], np.array([[[0.1, 0.9], [0.5, 0.5]], [[0.2, 0.8], [0.7, 0.3]]]))

# List of all factors in the network
factors = [A, B_given_A, C_given_A, D_given_BC]

# Query P(D|C=0)
query = ['D']
evidence = {'C': 0}

# Perform variable elimination
result = variable_elimination(factors, query, evidence)
print(result)


[0.47593583 0.52406417]


# code 4


In [11]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('B', 'A'), ('E', 'A'), ('A', 'D'), ('A', 'S')])

# Define the Conditional Probability Distributions (CPDs)
cpd_b = TabularCPD(variable='B', variable_card=2, values=[[0.99], [0.01]])
cpd_e = TabularCPD(variable='E', variable_card=2, values=[[0.98], [0.02]])
cpd_a = TabularCPD(variable='A', variable_card=2,
                   values=[[0.99, 0.06, 0.71, 0.05],
                           [0.01, 0.94, 0.29, 0.95]],
                   evidence=['B', 'E'], evidence_card=[2, 2])
cpd_d = TabularCPD(variable='D', variable_card=2,
                   values=[[0.95, 0.1],
                           [0.05, 0.9]],
                   evidence=['A'], evidence_card=[2])
cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.7, 0.2],
                           [0.3, 0.8]],
                   evidence=['A'], evidence_card=[2])

# Add CPDs to the model
model.add_cpds(cpd_b, cpd_e, cpd_a, cpd_d, cpd_s)

# Check if the model is consistent with the CPDs
assert model.check_model()

# Print the Conditional Probability Tables (CPTs)
print("CPT for B (Burglary):")
print(cpd_b)

print("\nCPT for E (Earthquake):")
print(cpd_e)

print("\nCPT for A (Alarm) given B (Burglary) and E (Earthquake):")
print(cpd_a)

print("\nCPT for D (David calls) given A (Alarm):")
print(cpd_d)

print("\nCPT for S (Sophia calls) given A (Alarm):")
print(cpd_s)

# Initialize the Variable Elimination inference
inference = VariableElimination(model)

# Computing the marginal probability of variables
marginal_prob_A = inference.query(variables=['A'])
marginal_prob_D = inference.query(variables=['D'])

print("\nMarginal Probability of ALARM (A):")
print(marginal_prob_A)

print("\nMarginal Probability of DAVID CALLS (D):")
print(marginal_prob_D)

CPT for B (Burglary):
+------+------+
| B(0) | 0.99 |
+------+------+
| B(1) | 0.01 |
+------+------+

CPT for E (Earthquake):
+------+------+
| E(0) | 0.98 |
+------+------+
| E(1) | 0.02 |
+------+------+

CPT for A (Alarm) given B (Burglary) and E (Earthquake):
+------+------+------+------+------+
| B    | B(0) | B(0) | B(1) | B(1) |
+------+------+------+------+------+
| E    | E(0) | E(1) | E(0) | E(1) |
+------+------+------+------+------+
| A(0) | 0.99 | 0.06 | 0.71 | 0.05 |
+------+------+------+------+------+
| A(1) | 0.01 | 0.94 | 0.29 | 0.95 |
+------+------+------+------+------+

CPT for D (David calls) given A (Alarm):
+------+------+------+
| A    | A(0) | A(1) |
+------+------+------+
| D(0) | 0.95 | 0.1  |
+------+------+------+
| D(1) | 0.05 | 0.9  |
+------+------+------+

CPT for S (Sophia calls) given A (Alarm):
+------+------+------+
| A    | A(0) | A(1) |
+------+------+------+
| S(0) | 0.7  | 0.2  |
+------+------+------+
| S(1) | 0.3  | 0.8  |
+------+------+---

# code - 5 cold,sneeze submiited code -- non edited --original

In [6]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([('Cold', 'Sneeze'), ('Cat', 'Allergy'), ('Allergy', 'Sneeze'), ('Allergy', 'Scratches')])

# Define the Conditional Probability Distributions (CPDs)
cpd_cold = TabularCPD(variable='Cold', variable_card=2, values=[[0.95], [0.05]])
cpd_cat = TabularCPD(variable='Cat', variable_card=2, values=[[0.98], [0.02]])
cpd_allergy = TabularCPD(variable='Allergy', variable_card=2,
                         values=[[0.95, 0.25],
                                 [0.05, 0.75]],
                         evidence=['Cat'], evidence_card=[2])
cpd_sneeze = TabularCPD(variable='Sneeze', variable_card=2,
                        values=[[0.99, 0.3, 0.2, 0.1],
                                [0.01, 0.7, 0.8, 0.9]],
                        evidence=['Cold', 'Allergy'], evidence_card=[2, 2])
cpd_scratches = TabularCPD(variable='Scratches', variable_card=2,
                           values=[[0.95, 0.5],
                                   [0.05, 0.5]],
                           evidence=['Allergy'], evidence_card=[2])

# Add CPDs to the model
model.add_cpds(cpd_cold, cpd_cat, cpd_allergy, cpd_sneeze, cpd_scratches)

# Check if the model is consistent with the CPDs
assert model.check_model()

# Print the CPTs
print("CPT of Cold:")
print(cpd_cold)
print("\nCPT of Cat:")
print(cpd_cat)
print("\nCPT of Allergy:")
print(cpd_allergy)
print("\nCPT of Sneeze:")
print(cpd_sneeze)
print("\nCPT of Scratches:")
print(cpd_scratches)

# Initialize the Variable Elimination inference
inference = VariableElimination(model)

# Computing the marginal probability of variables
marginal_prob_allergy = inference.query(variables=['Allergy'])
marginal_prob_sneeze = inference.query(variables=['Sneeze'])

print("\nMarginal Probability of Allergy:")
print(marginal_prob_allergy)
print("\nMarginal Probability of Sneeze:")
print(marginal_prob_sneeze)


CPT of Cold:
+---------+------+
| Cold(0) | 0.95 |
+---------+------+
| Cold(1) | 0.05 |
+---------+------+

CPT of Cat:
+--------+------+
| Cat(0) | 0.98 |
+--------+------+
| Cat(1) | 0.02 |
+--------+------+

CPT of Allergy:
+------------+--------+--------+
| Cat        | Cat(0) | Cat(1) |
+------------+--------+--------+
| Allergy(0) | 0.95   | 0.25   |
+------------+--------+--------+
| Allergy(1) | 0.05   | 0.75   |
+------------+--------+--------+

CPT of Sneeze:
+-----------+------------+------------+------------+------------+
| Cold      | Cold(0)    | Cold(0)    | Cold(1)    | Cold(1)    |
+-----------+------------+------------+------------+------------+
| Allergy   | Allergy(0) | Allergy(1) | Allergy(0) | Allergy(1) |
+-----------+------------+------------+------------+------------+
| Sneeze(0) | 0.99       | 0.3        | 0.2        | 0.1        |
+-----------+------------+------------+------------+------------+
| Sneeze(1) | 0.01       | 0.7        | 0.8        | 0.9       

## Same as code 5 ---- edited 

In [12]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


model = BayesianNetwork([('Cold', 'Sneeze'), ('Cat', 'Allergy'), ('Allergy', 'Sneeze'), ('Allergy', 'Scratches')])


cpd_cold = TabularCPD(variable='Cold', variable_card=2, values=[[0.95], [0.05]])
cpd_cat = TabularCPD(variable='Cat', variable_card=2, values=[[0.98], [0.02]])
cpd_allergy = TabularCPD(variable='Allergy', variable_card=2,
                         values=[[0.95, 0.25],
                                 [0.05, 0.75]],
                         evidence=['Cat'], evidence_card=[2])
cpd_sneeze = TabularCPD(variable='Sneeze', variable_card=2,
                        values=[[0.99, 0.3, 0.2, 0.1],
                                [0.01, 0.7, 0.8, 0.9]],
                        evidence=['Cold', 'Allergy'], evidence_card=[2, 2])
cpd_scratches = TabularCPD(variable='Scratches', variable_card=2,
                           values=[[0.95, 0.5],
                                   [0.05, 0.5]],
                           evidence=['Allergy'], evidence_card=[2])


model.add_cpds(cpd_cold, cpd_cat, cpd_allergy, cpd_sneeze, cpd_scratches)

assert model.check_model()

# Print the CPTs
print("CPT of Cold:")
print(cpd_cold)
print("\nCPT of Cat:")
print(cpd_cat)
print("\nCPT of Allergy:")
print(cpd_allergy)
print("\nCPT of Sneeze:")
print(cpd_sneeze)
print("\nCPT of Scratches:")
print(cpd_scratches)

inference = VariableElimination(model)

marginal_prob_allergy = inference.query(variables=['Allergy'])
marginal_prob_sneeze = inference.query(variables=['Sneeze'])

print("\nMarginal Probability of Allergy:")
print(marginal_prob_allergy)
print("\nMarginal Probability of Sneeze:")
print(marginal_prob_sneeze)


CPT of Cold:
+---------+------+
| Cold(0) | 0.95 |
+---------+------+
| Cold(1) | 0.05 |
+---------+------+

CPT of Cat:
+--------+------+
| Cat(0) | 0.98 |
+--------+------+
| Cat(1) | 0.02 |
+--------+------+

CPT of Allergy:
+------------+--------+--------+
| Cat        | Cat(0) | Cat(1) |
+------------+--------+--------+
| Allergy(0) | 0.95   | 0.25   |
+------------+--------+--------+
| Allergy(1) | 0.05   | 0.75   |
+------------+--------+--------+

CPT of Sneeze:
+-----------+------------+------------+------------+------------+
| Cold      | Cold(0)    | Cold(0)    | Cold(1)    | Cold(1)    |
+-----------+------------+------------+------------+------------+
| Allergy   | Allergy(0) | Allergy(1) | Allergy(0) | Allergy(1) |
+-----------+------------+------------+------------+------------+
| Sneeze(0) | 0.99       | 0.3        | 0.2        | 0.1        |
+-----------+------------+------------+------------+------------+
| Sneeze(1) | 0.01       | 0.7        | 0.8        | 0.9       